In [15]:
import requests
import sqlite3
import time
from datetime import datetime , date
from multiprocessing import Pool
import multiprocessing
import json
import pandas as pd
import pytz
desired_timezone = pytz.timezone('US/Eastern')
# Polygon API endpoint for retrieving currency pair prices
API_BASE_URL = 'https://api.polygon.io/v1'

# Your Polygon API key
API_KEY = 'beBybSi8daPgsTp5yx5cHtHpYcrjp5Jq'

# Database connection
DB_NAME = 'trades.db'


def create_database():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    # Create the average_prices table
    c.execute('''CREATE TABLE IF NOT EXISTS trades_sell
                 (currency_pair TEXT, trade TEXT, trade_price REAL, sold_units REAL, available_units REAL)''')
    c.execute('''CREATE TABLE IF NOT EXISTS trades_buy
                 (currency_pair TEXT, trade TEXT, trade_price REAL, purchased_units REAL, available_units REAL)''')
    c.execute('''CREATE TABLE IF NOT EXISTS close_sell
                 (currency_pair TEXT, trade TEXT, trade_price REAL, sold_units REAL, available_units REAL)''')
    c.execute('''CREATE TABLE IF NOT EXISTS close_buy
                 (currency_pair TEXT, trade TEXT, trade_price REAL, purchased_units REAL, available_units REAL)''')
    conn.commit()
    conn.close()

In [16]:

def get_currency_pairs():
    currency_pairs = []
    response = input("Define the currency pair trades (e.g., EURUSD_L, GBPCHF_S)")
    currency_pairs.append(response.strip().upper())
    return currency_pairs


In [17]:

def get_currency_pair_price(currency_pair):
    url = f"https://api.polygon.io/v1/conversion/{currency_pair[:3]}/{currency_pair[3:]}?amount=1&precision=4&apiKey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        try:
            data = response.json()
            ask = data['last']['ask']
            bid = data['last']['bid']
            price = (ask + bid) / 2
            return price
        except json.JSONDecodeError:
            print(f"Error decoding JSON for currency pair: {currency_pair}")
    else:
        print(f"Error retrieving data for currency pair: {currency_pair}. Status code: {response.status_code}")
    return None

In [18]:
def get_average_price(currency_pair,trade):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    if trade=='S':
      c.execute('SELECT trade_price FROM trades_sell WHERE currency_pair = ?',(currency_pair,))
      prices = c.fetchall()
      c.execute('SELECT total sold_units FROM trades_sell WHERE currency_pair = ?',(currency_pair,))
      executed_units = c.fetchall()
      price_sum = sum(price[0] for price in prices)
      executed_max = max(executed_unit[0] for executed_unit in executed_units)
      avg_price = price_sum / executed_max
      conn.commit()
      conn.close()
      return avg_price

    if trade=='L':
      c.execute('SELECT trade_price FROM trades_buy WHERE currency_pair = ?', (currency_pair,))
      prices = c.fetchall()
      c.execute('SELECT purchased_units FROM trades_buy WHERE currency_pair = ?', (currency_pair,))
      executed_units = c.fetchall()
      price_sum = sum(price[0] for price in prices)
      executed_max = max(executed_unit[0] for executed_unit in executed_units)
      avg_price = price_sum / len(executed_units)
      conn.commit()
      conn.close()
      return avg_price

In [19]:
def close_average_price(currency_pair,trade):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    if trade=='S':
      c.execute('SELECT trade_price FROM close_sell WHERE currency_pair = ?',(currency_pair,))
      prices = c.fetchall()
      c.execute('SELECT total sold_units FROM close_sell WHERE currency_pair = ?',(currency_pair,))
      executed_units = c.fetchall()
      price_sum = sum(price[0] for price in prices)
      executed_max = max(executed_unit[0] for executed_unit in executed_units)
      avg_price = price_sum / executed_max
      conn.commit()
      conn.close()
      return avg_price

    if trade=='L':
      c.execute('SELECT trade_price FROM close_buy WHERE currency_pair = ?', (currency_pair,))
      prices = c.fetchall()
      c.execute('SELECT purchased_units FROM close_buy WHERE currency_pair = ?', (currency_pair,))
      executed_units = c.fetchall()
      price_sum = sum(price[0] for price in prices)
      executed_max = max(executed_unit[0] for executed_unit in executed_units)
      avg_price = price_sum / len(executed_units)
      conn.commit()
      conn.close()
      return avg_price

In [20]:

def store_trade(currency_pair,trade_type,trade_price,executed_units,remaining_units):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    if trade_type=='S':
      c.execute('INSERT INTO trades_sell VALUES (?,?, ?, ?, ?)', (currency_pair,trade_type,trade_price,executed_units,remaining_units))
    elif trade_type=='L':
      c.execute('INSERT INTO trades_buy VALUES (?, ?,?, ?, ?)', (currency_pair,trade_type,trade_price,executed_units,remaining_units))
    conn.commit()
    conn.close()

def close_store_trade(currency_pair,trade_type,trade_price,executed_units,remaining_units):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    if trade_type=='S':
      c.execute('INSERT INTO close_buy VALUES (?,?, ?, ?, ?)', (currency_pair,trade_type,trade_price,executed_units,remaining_units))
    elif trade_type=='L':
      c.execute('INSERT INTO close_sell VALUES (?, ?,?, ?, ?)', (currency_pair,trade_type,trade_price,executed_units,remaining_units))
    conn.commit()
    conn.close()

In [21]:
def execute_trade(currency_pair, trade, execution_windows):
    interval = 1 * 60  # 1 minutes
    start_time_str = execution_windows[0]
    end_time_str = execution_windows[1]
    units = int(execution_windows[2])
    start_time_loc = f"{date.today()} {start_time_str}"
    end_time_loc = f"{date.today()} {end_time_str}"
    start_time = desired_timezone.localize(datetime.strptime(start_time_loc, "%Y-%m-%d %H:%M"))
    end_time = desired_timezone.localize(datetime.strptime(end_time_loc, "%Y-%m-%d %H:%M"))
    print(start_time)
    print(end_time)
    while datetime.now(desired_timezone) < start_time:
      time.sleep(1)
    while datetime.now(desired_timezone) >= start_time and datetime.now(desired_timezone) <= end_time:
      print(f"Executing trade for {currency_pair} at {datetime.now(desired_timezone)}")
      price = get_currency_pair_price(currency_pair)
      if trade=='S':
        selling_price = price*5
        max_units = units
        available_units=units
        sold_units=0
        if sold_units < max_units:
          sold_units = sold_units+5
          available_units=available_units-10
          store_trade(currency_pair,trade,selling_price,sold_units,available_units)
        else:
          break
        average_price = get_average_price(currency_pair,trade)
        print('The average price for the {} after this selling {} units is {}'.format(currency_pair,sold_units,average_price))
      elif trade=='L':
        purchased_units = 0
        max_units = units
        available_units = units
        if (purchased_units < max_units):
          buying_price = price*5
          purchased_units = purchased_units+5
          available_units = available_units-5
          store_trade(currency_pair,trade,buying_price,purchased_units,available_units)
        else:
          break
        average_price = get_average_price(currency_pair,trade)
        print('The average price for the {} after this selling {} units is {}'.format(currency_pair,sold_units,average_price))
      time.sleep(interval)

In [22]:
def get_execution_windows_and_units(currency):
    execution_windows = []
    print(f"Define the execution windows and execution units for {currency}")
    for i in range(1, 5):
        while True:
            response = input(f"Enter the execution window {i} in the format start_time-end_time-units (e.g., 10:24-20:00-20): ")
            parts = response.strip().split("-")
            start_time, end_time, units = parts[0], parts[1], parts[2]
            # Check if the new execution window overlaps with existing windows
            overlap = False
            for window in execution_windows:
                existing_start_time, existing_end_time, _ = window
                if start_time < existing_end_time and end_time > existing_start_time:
                    overlap = True
                    break     
            if overlap:
                print("The execution window overlaps with existing windows. Please try again.")
            else:
                execution_windows.append((start_time, end_time, units))
                break
    return execution_windows

In [23]:

def execute_closing_trades(currency,trade):
  currency_pair = currency
  interval = 1 * 60  # 1 minutes
  conn = sqlite3.connect(DB_NAME)
  c = conn.cursor()
  for i in range(0,5):
    price = get_currency_pair_price(currency_pair)
    if trade=='S':
      trade_type='L'
      #we will buy back all the sold trades
      c.execute('SELECT sold_units FROM trades_sell WHERE currency_pair = ?', (currency_pair,))
      executed_units = c.fetchall()
      executed_max = max(executed_unit[0] for executed_unit in executed_units)
      max_units= executed_max
      purchased_units = 0
      available_units = executed_units
      if (purchased_units < max_units):
        buying_price = price*5
        purchased_units = purchased_units+5
        available_units = available_units-5
        close_store_trade(currency_pair,trade_type,buying_price,purchased_units,available_units)
      else:
        break
      average_price = close_average_price(currency_pair,trade_type)
      print('The average price for the {} after this buying {} units is {} in closing trade'.format(currency_pair,purchased_units,average_price))
    elif trade=='L':
      trade_type='S'
    #we will sell all the purchased trades
    c.execute('SELECT purchased_units FROM trades_buy WHERE currency_pair = ?', (currency,))
    executed_units = c.fetchall()
    executed_max = max(executed_unit[0] for executed_unit in executed_units)
    sold_units = 0
    max_units = executed_units
    available_units = executed_units
    if (purchased_units < max_units):
      selling_price = price*5
      sold_units = sold_units+5
      available_units = available_units-5
      close_store_trade(currency_pair,trade_type,selling_price,sold_units,available_units)
    else:
      break
    average_price = close_average_price(currency_pair,trade_type)
    print('The average price for the {} after this selling {} units is {}'.format(currency_pair,sold_units,average_price))
  time.sleep(interval)


In [24]:

def main():
    currency_pairs = get_currency_pairs()
    currency,trade = currency_pairs[0].split('_')
    print("Selected currency pairs for trades are {0} for {1}".format(currency,trade))
    execution_windows = get_execution_windows_and_units(currency)
    print(execution_windows)
    for execution_window in execution_windows:
      execute_trade(currency,trade,execution_window)
    print(f"Executing closing trades for {currency}...")
    execute_closing_trades(currency,trade)

if __name__ == '__main__':
    create_database()
    main()


Selected currency pairs for trades are EURUSD for L
Define the execution windows and execution units for EURUSD
[('17:50', '17:52', '20'), ('17:53', '17:55', '30'), ('17:56', '17:58', '30'), ('17:59', '18:01', '20')]
2023-07-14 17:50:00-04:00
2023-07-14 17:52:00-04:00
Executing trade for EURUSD at 2023-07-14 17:50:00.117846-04:00


UnboundLocalError: local variable 'sold_units' referenced before assignment